# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [ ]:
import pandas as pd
file1="https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv"
file2="https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv"
file3="https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv"
def clean_df(df):
    df.columns=df.columns.str.lower()
    df.columns=df.columns.str.replace(" ","_")
    df.columns=df.columns.str.replace("custateomer","customer")
    def gender_updating(x):
        if x in ["F", "Femal", "female"]:
            return "F"
        elif x in ["M", "Male"]:
            return "M"
    df["gender"] = df["gender"].apply(gender_updating)
    df["state"]=df["state"].replace({"Cali":"California","WA":"Washington","AZ":"Arizona"})
    df["education"]=df["education"].replace("Bachelor","Bachelors")
    df["customer_lifetime_value"]=df["customer_lifetime_value"].str.replace("%","",regex=False)
    df["vehicle_class"]=df["vehicle_class"].replace({"Sports Car":"Luxury","Luxury SUV":"Luxury","Luxury Car":"Luxury"})
    df['number_of_open_complaints'] = (
    df['number_of_open_complaints']
    .astype(str)                     
    .str.split('/')                  
    .str[1]                          
    .astype(float))                   
    df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')
    df.fillna(method="ffill",inplace=True)
    df.gender.fillna(df["gender"].mode()[0],inplace=True)
    df["customer_lifetime_value"].fillna(df['customer_lifetime_value'].mean(), inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df
files = [file1, file2, file3]
df_list = [clean_df(pd.read_csv(f)) for f in files]
combined = pd.concat(df_list, ignore_index=True)

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [34]:
new_df=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv")
new_df.columns

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [39]:
new_df.pivot_table(index="sales_channel",values=["total_claim_amount"],aggfunc="sum").round(2)

,total_claim_amount
sales_channel,
Agent,1810226.82
Branch,1301204.00
Call Center,926600.82
Web,706600.04


In [38]:
new_df.pivot_table(index="gender",columns="education",values=["customer_lifetime_value"],aggfunc="mean")

customer_lifetime_value                            \
education                Bachelor      College       Doctor   
gender                                                        
F                     7874.269478  7748.823325  7328.508916   
M                     7703.601675  8052.459288  7415.333638   

                                             
education High School or Below       Master  
gender                                       
F                  8675.220201  8157.053154  
M                  8149.687783  8168.832659

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [47]:
summary = new_df.groupby(['policy_type', 'month'])['number_of_open_complaints'].count().reset_index()

In [48]:
summary

,policy_type,month,number_of_open_complaints
0,Corporate Auto,1,1252
1,Corporate Auto,2,1089
2,Personal Auto,1,4329
3,Personal Auto,2,3799
4,Special Auto,1,237
5,Special Auto,2,204
